# 머지 소트 트리 (Merge Sort Tree)

## 수열과 쿼리 3

- 문제 출처: [백준 13544번](https://www.acmicpc.net/problem/13544)

`-` 머지 소트 트리에 대해 알아보는 문제이다

`-` 내가 알고 있는 건 머지 소트뿐이다

`-` 이거랑 무슨 상관일까

`-` 부분 수열 중에서 $k$보다 큰 원소의 개수를 세야한다

`-` 모든 원소를 순회하면 $O(N)$이며 쿼리가 $M$개이므로 전체 알고리즘의 시간 복잡도는 $O(NM)$이다

`-` 당연히 시간 초과이다

`-` 만약 부분 수열이 정렬되어 있다면 $k$보다 큰 원소의 개수를 이분 탐색으로 $O(\log N)$에 계산할 수 있다

`-` 그럴러면 세그먼트 트리의 노드별로 담당 구간의 원소를 정렬한 상태로 가지고 있어야 한다

`-` 잘 생각해보면 트리의 높이는 $O(\log N)$이고 각 높이마다 $N$개의 원소를 가지고 있으니 전체 원소의 개수는 $O(N\log N)$이다 (저장하기에 충분)

`-` 그럼 트리를 구축할 때 원소를 정렬하기만 하면 쿼리를 $O\left(\log^2 N\right)$에 수행할 수 있다 (완전히 포함되는 노드의 수는 $O(\log N)$이다)

`-` 왼쪽 자식의 정렬된 원소와 오른쪽 자식의 정렬된 원소가 있을 때 머지 소트에서의 병합 과정을 그대로 수행하면 둘을 합치며 정렬할 수 있다

`-` 트리의 각 높이마다 $O(N)$의 시간 복잡도를 가지므로 머지 소트 트리를 구축하는 건 $O(N\log N)$의 시간 복잡도를 가진다

`-` 이제 원하는 구간에 속하는 노드를 찾고 각각에 대해 $k$보다 큰 원소를 이분 탐색으로 센 뒤 개수를 전부 더해주면 된다

`-` 배열의 크기가 $n$이고 $k$보다 큰 원소가 처음 등장하는 인덱스가 $i$라면 $k$보다 큰 원소의 개수는 $n - i$이다

`-` 만약 배열의 모든 원소가 $k$보다 작거나 같으면 $i=n$이므로 $n-i=0$이 된다

`-` 전체 알고리즘의 시간 복잡도는 $O\left(N \log N + M \log^2 N\right)$이다

`-` 머지 소트 트리라는 용어를 알고 시작해서 해결할 수 있었다 (이름이 정말 직관적이다)

In [16]:
import math


def merge(left, right):
    sorted_array = []
    n, m = len(left), len(right)
    i, j = 0, 0
    while i < n and j < m:
        if left[i] < right[j]:
            sorted_array.append(left[i])
            i += 1
        else:
            sorted_array.append(right[j])
            j += 1
    while i < n:
        sorted_array.append(left[i])
        i += 1
    while j < m:
        sorted_array.append(right[j])
        j += 1
    return sorted_array


def init(array, tree, node, start, end):
    if start == end:
        tree[node] = [array[start]]
        return
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    init(array, tree, left_child, start, mid)
    init(array, tree, right_child, mid + 1, end)
    tree[node] = merge(tree[left_child], tree[right_child])


def binary_search(array, k):
    n = len(array)
    left = 0
    right = n - 1
    while left <= right:
        mid = (left + right) // 2
        if array[mid] <= k:
            left = mid + 1
        else:
            right = mid - 1
    return n - left


def query(left, right, k, tree, node, start, end):
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        return binary_search(tree[node], k)
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    left_count = query(left, right, k, tree, left_child, start, mid)
    right_count = query(left, right, k, tree, right_child, mid + 1, end)
    return left_count + right_count


def solution():
    N = int(input())
    array = list(map(int, input().split()))
    M = int(input())
    h = math.ceil(math.log2(N))
    tree_size = 2**(h + 1)
    tree = [0] * tree_size
    start, end = 0, N - 1
    root = 1
    init(array, tree, root, start, end)
    last_ans = 0
    answers = []
    for _ in range(M):
        a, b, c = map(int, input().split())
        i, j, k = a ^ last_ans, b ^ last_ans, c ^ last_ans
        last_ans = query(i - 1, j - 1, k, tree, root, start, end)
        answers.append(last_ans)
    print("\n".join(map(str, answers)))


solution()

# input
# 5
# 5 1 2 3 4
# 3
# 2 4 1
# 6 6 6
# 1 5 2

 5
 5 1 2 3 4
 3
 2 4 1
 6 6 6
 1 5 2


2
0
3


`-` 이 문제를 풀었으면 [수열과 쿼리 1](https://www.acmicpc.net/problem/13537) 문제를 날먹할 수 있다

## 트리와 색깔

- 문제 출처: [백준 15899번](https://www.acmicpc.net/problem/15899)

`-` 오일러 투어 테크닉으로 트리를 $1$차원 배열로 만든 뒤 머지 소트 트리를 구축하면 된다

`-` 질의는 이분 탐색을 통해 $O\left(\log^2 N\right)$에 수행할 수 있다

`-` 질의는 노드별로 독립이므로 단순히 결괏값을 합산하면 된다

`-` 인덱스 처리에 주의하자

`-` 시간 초과를 너무 당해서 최적화를 수행했다

`-` 그래프를 딕셔너리가 아닌 배열로 관리했다

`-` 구간 쿼리를 스택을 이용하여 비재귀로 구현했다

`-` 배열의 마지막 원소가 $c$ 이하이거나 첫 번째 원소가 $c$를 초과하면 이분 탐색을 진행하지 않고 답을 반환하게 했다

`-` 배열의 값을 변경하지 않는다면 튜플을 사용하도록 했다 (세그먼트 트리의 리프 노드와 색깔 배열)

`-` 누적된 개수가 $10^9 + 7$보다 클 때만 나머지 연산을 수행했다

`-` $1784 \operatorname{ms}$로 간신히 통과했다

`-` 웃긴게 정답 코드를 그대로 제출하니까 시간 초과를 받았다

`-` 이 문제를 맞힌 건 기적이다

In [68]:
import math
import sys
from collections import defaultdict

sys.setrecursionlimit(2 * 10**5)


def dfs(graph, node, parent, array, colors, query_ranges):
    global NTH
    array[NTH] = colors[node - 1]
    query_ranges[node][START] = NTH
    for child in graph[node]:
        if child == parent:
            continue
        NTH += 1
        dfs(graph, child, node, array, colors, query_ranges)
    query_ranges[node][END] = NTH


def merge(left, right):
    sorted_array = []
    n, m = len(left), len(right)
    i = j = 0
    while i < n and j < m:
        if left[i] < right[j]:
            sorted_array.append(left[i])
            i += 1
        else:
            sorted_array.append(right[j])
            j += 1
    sorted_array.extend(left[i:])
    sorted_array.extend(right[j:])
    return sorted_array


def init(array, tree, node, start, end):
    if start == end:
        tree[node] = (array[start],)
        return
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    init(array, tree, left_child, start, mid)
    init(array, tree, right_child, mid + 1, end)
    tree[node] = merge(tree[left_child], tree[right_child])


def binary_search(array, value):
    left, right = 0, len(array) - 1
    while left <= right:
        mid = (left + right) // 2
        if array[mid] <= value:
            left = mid + 1
        else:
            right = mid - 1
    return left


def range_query(left, right, c, tree, node, start, end):
    count = 0
    stack = [(node, start, end)]
    while stack:
        node, start, end = stack.pop()
        if left <= start and end <= right:
            if tree[node][-1] <= c:
                count += end - start + 1
                continue
            if tree[node][0] > c:
                continue
            count += binary_search(tree[node], c)
            continue
        mid = (start + end) // 2
        if left <= mid and right >= start:
            stack.append((2 * node, start, mid))
        if left <= end and right >= mid + 1:
            stack.append((2 * node + 1, mid + 1, end))
    return count


def solution():
    global START, END, NTH
    N, M, C = map(int, input().split())
    mod = 10**9 + 7
    colors = tuple(map(int, input().split()))
    graph = [[] for _ in range(N + 1)]
    for _ in range(N - 1):
        u, v = map(int, input().split())
        graph[u].append(v)
        graph[v].append(u)
    START, END = 0, 1
    NTH = 0
    root, none = 1, -1
    array = [0] * N
    query_ranges = [[0] * 2 for _ in range(N + 1)]
    dfs(graph, root, none, array, colors, query_ranges)
    h = math.ceil(math.log2(N))
    tree_size = 2**(h + 1)
    tree = [0] * tree_size
    start, end = 0, N - 1
    init(array, tree, root, start, end)
    answer = 0
    for _ in range(M):
        v, c = map(int, input().split())
        left, right = query_ranges[v]
        count = range_query(left, right, c, tree, root, start, end)
        answer += count
        if answer < mod:
            continue
        answer %= mod
    print(answer)


solution()

# input
# 4 2 2
# 1 2 2 2
# 1 2
# 1 3
# 1 4
# 1 1
# 1 2

 4 2 2
 1 2 2 2
 1 2
 1 3
 1 4
 1 1
 1 2


5


`-` 다시 제출하니까 이번엔 맞혔다!

`-` 그런데 메모리 사용량이 다르다 (신기)